In [ ]:
%pip install selenium

In [ ]:
# 오늘의 경기 승률 예측 코드

In [ ]:
# 4-1. 경기 정보 데이터 (팀, 구장, 날짜, 시간) 스크래핑
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Chrome 드라이버를 새로 설치하여 Chrome 실행
driver = webdriver.Chrome()

# 사이트 접속
driver.get("https://www.koreabaseball.com/Schedule/Schedule.aspx")

# 페이지가 로드될 때까지 대기
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
time.sleep(2)

# 텍스트가 '프리뷰'인 a태그 추출
# 해당 사이트는 동적 페이지이기 때문에 동적 웹 스크래핑 라이브러리인 selenium을 사용해야 함
a_tag_preview_text = driver.find_elements(By.XPATH, "//a[normalize-space(text())='프리뷰']")


# URL을 저장할 빈 리스트 생성
preview_url_list = []

# 프리뷰 URL 추출
for index, link in enumerate(a_tag_preview_text):
    href = link.get_attribute('href')
    preview_url_list.append(href)

# Chrome 닫기
driver.quit()

# 출력
print(preview_url_list)

In [ ]:
# 4-2. 팀 기록 데이터 (순위, 팀명, 타율, 경기, 타석, 타수, 득점, 안타, 2루타, 3루타, 홈런, 투타, 타점, 희생번트, 희생플라이) 스크래핑

import requests
import pandas as pd
from bs4 import BeautifulSoup

url = "https://www.koreabaseball.com/Record/Team/Hitter/Basic1.aspx"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36"
}

# url로 html을 paersing 함, headers는 사용자가 사람이라는 것을 인증하기 위함
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

# <td> 태그 전체 수집
td_tags = soup.find_all('td')

# td에서 텍스트만 추출
td_text = [td.get_text(strip=True) for td in td_tags]

# 열 개수 (한 팀당 데이터 수)
# 순위 + 팀명 + AVG + G + PA + AB + R + H + 2B + 3B + HR + TB + RBI + SAC + SF
num_columns = 15

# td 리스트를 행 단위로 잘라서 리스트로 변환
data = [td_text[i:i + num_columns] for i in range(0, len(td_text), num_columns)]

# 열 이름 설정
columns = ['순위', '팀명', '타율', '경기', '타석', '타수', '득점', '안타', '2루타', '3루타', '홈런', '투타', '타점', '희생번트', '희생플라이']

# 데이터프레임 생성
team_info_df = pd.DataFrame(data, columns=columns)
team_info_df = team_info_df[team_info_df['순위'] != '합계']

# 출력
team_info_df

,순위,팀명,타율,경기,타석,타수,득점,안타,2루타,3루타,홈런런,투타,타점,희생번트,희생플라이
0,1,LG,0.281,18,711,598,112,168,28,1,18,252,101,9,8
1,2,롯데,0.271,20,783,679,84,184,35,3,8,249,81,12,5
2,3,NC,0.266,16,620,533,79,142,22,4,17,223,77,7,8
3,4,삼성,0.259,19,746,653,110,169,33,3,21,271,97,5,6
4,5,키움,0.255,20,779,678,93,173,32,1,18,261,84,5,6
5,6,두산,0.253,19,741,653,87,165,31,5,10,236,77,9,7
6,7,KIA,0.243,18,692,604,82,147,34,0,20,241,81,3,3
7,8,KT,0.234,18,683,593,59,139,28,2,7,192,57,6,7
8,9,SSG,0.231,16,617,527,59,122,15,1,8,163,58,13,6
9,10,한화,0.225,20,755,667,75,150,30,2,10,214,71,7,4
